# **GPT Settings**

In [1]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 2.9.0
    Uninstalling openai-2.9.0:
      Successfully uninstalled openai-2.9.0


In [2]:
import openai
import os

In [4]:
with open('/content/handshake_hex.txt', 'r') as f:  # Adjust path if needed
    hex_dump = f.read()

In [5]:
import re
# Extract hex bytes using regex
hex_lines = []
for line in hex_dump.strip().split('\n'):
    # Extract only valid 2-digit hex groups
    hex_bytes = re.findall(r'\b[0-9a-fA-F]{2}\b', line)
    if hex_bytes:
        hex_lines.append(' '.join(hex_bytes))

# Join all hex lines into one full hex stream
clean_hex_output = '\n'.join(hex_lines)
print(clean_hex_output)

02 00 00 00 45 00 00 34 6d 36 40 00 80 06 00 00
7f 00 00 01 7f 00 00 01 ca 3b 30 39 38 25 3e 79
00 00 00 00 80 02 ff ff 05 d7 00 00 02 04 ff d7
01 03 03 08 01 01 04 02
02 00 00 00 45 00 00 34 6d 37 40 00 80 06 00 00
7f 00 00 01 7f 00 00 01 30 39 ca 3b 57 dc 1f 22 09
38 25 3e 7a 80 12 ff ff 8e c7 00 00 02 04 ff d7
01 03 03 08 01 01 04 02
02 00 00 00 45 00 00 28 6d 38 40 00 80 06 00 00
7f 00 00 01 7f 00 00 01 ca 3b 30 39 38 25 3e 7a
57 dc 1f 23 50 10 00 ff c8 bf 00 00
02 00 00 00 45 00 00 35 6d 39 40 00 80 06 00 00
7f 00 00 01 7f 00 00 01 ca 3b 30 39 38 25 3e 7a
57 dc 1f 23 50 18 00 ff 58 6a 00 00 48 65 6c 6c
6f 20 53 65 72 76 65 72 21
02 00 00 00 45 00 00 28 6d 3a 40 00 80 06 00 00
7f 00 00 01 7f 00 00 01 30 39 ca 3b 57 dc 1f 23 09
38 25 3e 87 50 10 00 ff c8 b2 00 00
02 00 00 00 45 00 00 35 6d 3b 40 00 80 06 00 00
7f 00 00 01 7f 00 00 01 30 39 ca 3b 57 dc 1f 23 09
38 25 3e 87 50 18 00 ff 68 65 00 00 48 65 6c 6c
6f 20 43 6c 69 65 6e 74 21
02 00 00 00 45 00 00 28 6d 3c 40 00 80 06 00 00
7

# **1. Ground Truth**

In [11]:
import re
import pandas as pd

rows = []

with open("ground_truth.txt", "r") as f:
    for line in f:
        line = line.strip()
        if not line or "TCP" not in line:
            continue

        # Packet ID and timestamp
        m = re.match(r"(\d+)\s+([\d\.]+)\s+(.*)", line)
        if not m:
            continue

        packet_id = int(m.group(1))
        time = float(m.group(2))
        rest = m.group(3)

        # IPs and ports
        ip_match = re.search(
            r"(\d+\.\d+\.\d+\.\d+)\s+→\s+(\d+\.\d+\.\d+\.\d+).*?(\d+)\s+→\s+(\d+)",
            rest
        )
        if not ip_match:
            continue

        src_ip, dst_ip, src_port, dst_port = ip_match.groups()

        # TCP flags
        flags_match = re.search(r"\[(.*?)\]", rest)
        flags = flags_match.group(1) if flags_match else ""

        # Normalise packet type
        if flags == "SYN":
            pkt_type = "SYN"
        elif flags == "SYN, ACK":
            pkt_type = "SYN-ACK"
        elif flags == "ACK":
            pkt_type = "ACK"
        elif "FIN" in flags:
            pkt_type = "FIN"
        else:
            pkt_type = "OTHER"

        # Sequence / Ack / Win / Len
        seq = re.search(r"Seq=(\d+)", rest)
        ack = re.search(r"Ack=(\d+)", rest)
        win = re.search(r"Win=(\d+)", rest)
        length = re.search(r"Len=(\d+)", rest)
        ws = re.search(r"WS=(\d+)", rest)
        mss = re.search(r"MSS=(\d+)", rest)

        rows.append({
            "packet_id": packet_id,
            "time": time,
            "src_ip": src_ip,
            "dst_ip": dst_ip,
            "src_port": int(src_port),
            "dst_port": int(dst_port),
            "tcp_flags": flags,
            "pkt_type": pkt_type,
            "seq": int(seq.group(1)) if seq else None,
            "ack": int(ack.group(1)) if ack else None,
            "win": int(win.group(1)) if win else None,
            "len": int(length.group(1)) if length else None,
            "ws": int(ws.group(1)) if ws else None,
            "mss": int(mss.group(1)) if mss else None,
        })

df_gt = pd.DataFrame(rows)
df_gt.head()


,packet_id,time,src_ip,dst_ip,src_port,dst_port,tcp_flags,pkt_type,seq,ack,win,len,ws,mss
0,1,0.000000,127.0.0.1,127.0.0.1,51771,12345,SYN,SYN,0,NaN,65535,0,256.0,65495.0
1,2,0.000072,127.0.0.1,127.0.0.1,12345,51771,"SYN, ACK",SYN-ACK,0,1.0,65535,0,256.0,65495.0
2,3,0.000107,127.0.0.1,127.0.0.1,51771,12345,ACK,ACK,1,1.0,65280,0,NaN,NaN
3,4,0.000361,127.0.0.1,127.0.0.1,51771,12345,"PSH, ACK",OTHER,1,1.0,65280,13,NaN,NaN
4,5,0.000382,127.0.0.1,127.0.0.1,12345,51771,ACK,ACK,1,14.0,65280,0,NaN,NaN


# **2. Evaluation**

In [ ]:
openai.api_key = os.getenv("sk-.....")

MODELS = ["gpt-3.5-turbo", "gpt-4.1"]
PROMPTING = ["baseline", "oneshot"]
N_REPETITIONS = 20


In [13]:
ONE_SHOT_EXAMPLE = """
TCP Packet Type,Source Port,Destination Port,Sequence Number,Acknowledgement Number,Data Offset,Reserved,TCP Flags,Window Size,Checksum,Urgent Pointer
SYN,51771,12345,0,0,8,0,SYN,65535,0x05d7,0
""".strip()


In [14]:
def build_prompt(hex_packet: str, mode: str) -> str:
    if mode == "baseline":
        return f"""
You are given ONE TCP packet as a raw hex dump.
The packet includes Linux cooked capture + IPv4 + TCP.

Extract ONLY the TCP header fields.

Fields:
- TCP Packet Type (SYN, SYN-ACK, ACK, FIN, RST, OTHER)
- Source Port
- Destination Port
- Sequence Number
- Acknowledgement Number
- Data Offset
- Reserved
- TCP Flags
- Window Size
- Checksum
- Urgent Pointer

Input hex:
{hex_packet}

Return ONE CSV row with this header:
TCP Packet Type,Source Port,Destination Port,Sequence Number,Acknowledgement Number,Data Offset,Reserved,TCP Flags,Window Size,Checksum,Urgent Pointer

Output only CSV.
""".strip()

    elif mode == "oneshot":
        return f"""
You are given ONE TCP packet as a raw hex dump.

Example (format only):
{ONE_SHOT_EXAMPLE}

Now extract TCP header fields from the packet below.
Think step by step internally. Do not show reasoning.

Input hex:
{hex_packet}

Return ONE CSV row with this header:
TCP Packet Type,Source Port,Destination Port,Sequence Number,Acknowledgement Number,Data Offset,Reserved,TCP Flags,Window Size,Checksum,Urgent Pointer

Output only CSV.
""".strip()


In [15]:
def call_gpt(prompt: str, model: str) -> str:
    resp = openai.ChatCompletion.create(
        model=model,
        temperature=0.2,
        messages=[
            {"role": "system", "content": "You extract TCP headers into CSV."},
            {"role": "user", "content": prompt},
        ],
    )
    return resp.choices[0].message["content"]


In [16]:
def parse_csv_output(text):
    lines = [l for l in text.splitlines() if l and not l.startswith("```")]
    return pd.read_csv(StringIO("\n".join(lines)))


In [17]:
FIELDS = ["src_port", "dst_port", "seq", "ack", "tcp_flags", "win"]

def evaluate_packet(gt_row, pred_row):
    total = len(FIELDS)
    correct = 0
    missing = 0

    for f in FIELDS:
        if f not in pred_row or pd.isna(pred_row[f]):
            missing += 1
        elif str(gt_row[f]).strip() == str(pred_row[f]).strip():
            correct += 1

    return {
        "missing_rate": missing / total,
        "correct_rate": correct / total
    }


In [27]:
hex_flat = clean_hex_output.replace("\n", " ").strip()
hex_flat = " ".join(hex_flat.split())


In [28]:
import re

packets = re.split(r'(?=02 00 00 00 45)', hex_flat)


In [29]:
packets = [
    p.replace(" ", "")        # remove spaces
    for p in packets
    if p.strip()              # remove empties
]


In [30]:
import pandas as pd

df_hex = pd.DataFrame({
    "packet_id": range(1, len(packets) + 1),
    "hex_clean": packets
})

df_hex.head()


,packet_id,hex_clean
0,1,02000000450000346d364000800600007f0000017f0000...
1,2,02000000450000346d374000800600007f0000017f0000...
2,3,02000000450000286d384000800600007f0000017f0000...
3,4,02000000450000356d394000800600007f0000017f0000...
4,5,02000000450000286d3a4000800600007f0000017f0000...


In [31]:
df_task = df_gt.merge(df_hex, on="packet_id", how="inner")


In [32]:
print("Total packets:", len(df_hex))
print("Example packet length:", len(df_hex.loc[0, "hex_clean"]))
print(df_hex.loc[0, "hex_clean"][:40])


Total packets: 221
Example packet length: 112
02000000450000346d364000800600007f000001


In [37]:
df_task.head()

,packet_id,time,src_ip,dst_ip,src_port,dst_port,tcp_flags,pkt_type,seq,ack,win,len,ws,mss,hex_clean
0,1,0.000000,127.0.0.1,127.0.0.1,51771,12345,SYN,SYN,0,NaN,65535,0,256.0,65495.0,02000000450000346d364000800600007f0000017f0000...
1,2,0.000072,127.0.0.1,127.0.0.1,12345,51771,"SYN, ACK",SYN-ACK,0,1.0,65535,0,256.0,65495.0,02000000450000346d374000800600007f0000017f0000...
2,3,0.000107,127.0.0.1,127.0.0.1,51771,12345,ACK,ACK,1,1.0,65280,0,NaN,NaN,02000000450000286d384000800600007f0000017f0000...
3,4,0.000361,127.0.0.1,127.0.0.1,51771,12345,"PSH, ACK",OTHER,1,1.0,65280,13,NaN,NaN,02000000450000356d394000800600007f0000017f0000...
4,5,0.000382,127.0.0.1,127.0.0.1,12345,51771,ACK,ACK,1,14.0,65280,0,NaN,NaN,02000000450000286d3a4000800600007f0000017f0000...


In [39]:
df_task.to_csv("ground_truth_hex.csv", index=False)

In [ ]:
import os, time
import pandas as pd
import openai
from io import StringIO

openai.api_key = os.getenv("sk-................")

MODELS = ["gpt-4.1", "gpt-3.5-turbo"]
PROMPTING = ["baseline", "oneshot"]
N_REPETITIONS = 20


In [41]:
# model CSV column -> ground truth column
FIELDS_TO_COMPARE = {
    "Source Port": "src_port",
    "Destination Port": "dst_port",
    "Sequence Number": "seq",
    "Acknowledgement Number": "ack",
    "TCP Flags": "tcp_flags",
}


In [42]:
SYN_ONE_SHOT_EXAMPLE = """
TCP Packet Type,Source Port,Destination Port,Sequence Number,Acknowledgement Number,Data Offset,Reserved,TCP Flags,Window Size,Checksum,Urgent Pointer
SYN,51771,12345,0,0,8,0,SYN,65535,0x0000,0
""".strip()

def build_prompt_baseline(hex_pkt):
    return f"""
Extract TCP header fields from the raw hex dump.

Hex:
{hex_pkt}

Return CSV only with header:
TCP Packet Type,Source Port,Destination Port,Sequence Number,Acknowledgement Number,Data Offset,Reserved,TCP Flags,Window Size,Checksum,Urgent Pointer
""".strip()

def build_prompt_oneshot(hex_pkt):
    return f"""
Extract TCP header fields from the raw hex dump.

ONE-SHOT EXAMPLE:
{SYN_ONE_SHOT_EXAMPLE}

Now parse this packet.

Hex:
{hex_pkt}

Think step-by-step internally. Do not explain.

Return CSV only.
""".strip()

def build_prompt(hex_pkt, mode):
    return build_prompt_baseline(hex_pkt) if mode == "baseline" else build_prompt_oneshot(hex_pkt)


In [43]:
def evaluate_against_gt(gt_df, model_df):
    rows = []

    gt = gt_df.copy()
    md = model_df.copy()

    gt["pkt_type"] = gt["pkt_type"].str.upper()
    md["TCP Packet Type"] = md["TCP Packet Type"].str.upper()

    for pkt_type in gt["pkt_type"].unique():
        gt_rows = gt[gt["pkt_type"] == pkt_type].reset_index(drop=True)
        md_rows = md[md["TCP Packet Type"] == pkt_type].reset_index(drop=True)

        n = min(len(gt_rows), len(md_rows))
        if n == 0:
            continue

        missing = correct = wrong = 0

        for i in range(n):
            g = gt_rows.iloc[i]
            m = md_rows.iloc[i]

            for model_col, gt_col in FIELDS_TO_COMPARE.items():
                if model_col not in m or pd.isna(m[model_col]):
                    missing += 1
                elif str(m[model_col]).strip() == str(g[gt_col]).strip():
                    correct += 1
                else:
                    wrong += 1

        denom = n * len(FIELDS_TO_COMPARE)

        rows.append({
            "Packet Type": pkt_type,
            "Average Number of Missing Fields": missing / n,
            "Average Number of Correct Field Positions": correct / denom,
            "Average Number of Missing Field Value Pairs": (missing + wrong) / denom,
            "Number of Fields Extracted": len(md_rows) * len(FIELDS_TO_COMPARE),
        })

    return pd.DataFrame(rows)


In [ ]:
import openai
client=openai.api_key = "sk-................."
resp = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Say OK"}]
)

print(resp.choices[0].message["content"])


OK


In [60]:
# ============================
# TCP HEX PARSING EVAL HARNESS
# (df_task already in memory)
# ============================

import os, time, re
import pandas as pd
from io import StringIO

# -------------------------
# CONFIG
# -------------------------
MODELS = ["gpt-4.1", "gpt-3.5-turbo"]      # edit if needed
PROMPTING = ["baseline", "oneshot"]       # baseline / oneshot
N_REPETITIONS = 3                         # start small; 20 will be VERY slow

MAX_PACKETS = 30                          # start 10/30/50; scale later
BATCH_SIZE = 10                           # key speedup: 10 packets per call
SLEEP_SEC = 0.2                           # rate-limit cushion
TEMPERATURE = 0.0                         # if you want variance across runs, set 0.7

RAW_LOG_CSV = "raw_calls_log.csv"
METRICS_CSV = "metrics_llm_prompting.csv"
SUMMARY_CSV = "metrics_summary_llm_prompting.csv"



# -------------------------
# EXPECTED OUTPUT COLUMNS
# -------------------------
OUT_COLS = [
    "packet_id",
    "TCP Packet Type",
    "Source Port",
    "Destination Port",
    "Sequence Number",
    "Acknowledgement Number",
    "Data Offset",
    "Reserved",
    "TCP Flags",
    "Window Size",
    "Checksum",
    "Urgent Pointer",
]

# Compare these GT columns -> predicted columns
FIELDS_TO_COMPARE = {
    "src_port": "Source Port",
    "dst_port": "Destination Port",
    "seq": "Sequence Number",
    "ack": "Acknowledgement Number",
    "tcp_flags": "TCP Flags",
    "pkt_type": "TCP Packet Type",
}

# -------------------------
# ONE-SHOT FORMAT REFERENCE
# -------------------------
SYN_ONE_SHOT_GRAMMAR = """
{
  "TCP_SYN": {
    "Source_Port": "<Value>",
    "Destination_Port": "<Value>",
    "Sequence_Number": "<Value>",
    "Acknowledgment_Number": "00 00 00 00",
    "Data_Offset": "<Value>",
    "Reserved": "<Value>",
    "Flags": "02",
    "Window_Size": "<Value>",
    "Checksum": "<Value>",
    "Urgent_Pointer": "00 00",
    "TCP_Options": "<Value>",
    "Payload": "00"
  }
}
""".strip()

# -------------------------
# HELPERS
# -------------------------
def normalize_hex(hex_str: str) -> str:
    """Make hex one-line, no spaces/newlines, lowercase."""
    if pd.isna(hex_str):
        return ""
    s = str(hex_str).strip()
    s = re.sub(r"\s+", "", s)          # remove whitespace
    s = s.replace("0x", "")
    return s.lower()

def norm_flags(s: str) -> str:
    if pd.isna(s) or s is None:
        return ""
    s = str(s).upper().replace(",", "|").replace(" ", "")
    # common normalizations
    s = s.replace("SYNACK", "SYN|ACK")
    s = s.replace("PSHACK", "PSH|ACK")
    s = s.replace("FINACK", "FIN|ACK")
    return s

def build_prompt_batch(packet_ids, hex_list, prompting: str) -> str:
    """
    Ask the model to parse multiple packets at once.
    Must return strict CSV with packet_id column.
    """
    items = []
    for pid, hx in zip(packet_ids, hex_list):
        items.append(f"packet_id={pid}\nhex={hx}")
    block = "\n\n".join(items)

    header = ",".join(OUT_COLS)

    base_rules = f"""
You extract TCP header fields from raw hex dumps.

You will be given MULTIPLE packets. Each packet has:
- packet_id
- hex (a single packet hex string)

Return STRICT CSV with this exact header:
{header}

Rules:
- Output ONLY CSV (header + rows). No markdown, no explanations.
- One row per packet_id (same order is fine).
- TCP Flags must be text like SYN, SYN|ACK, ACK, PSH|ACK, FIN|ACK etc.
- "TCP Packet Type" must be one of: SYN, SYN-ACK, ACK, OTHER, UNKNOWN
""".strip()

    if prompting == "baseline":
        prompt = f"""{base_rules}

Packets:
{block}
""".strip()
    elif prompting == "oneshot":
        prompt = f"""{base_rules}

ONE-SHOT FORMAT REFERENCE (structure only):
{SYN_ONE_SHOT_GRAMMAR}

Packets:
{block}
""".strip()
    else:
        raise ValueError("Unknown prompting mode")

    return prompt

def call_llm(prompt: str, model_name: str, max_retries: int = 6) -> str:
    last_err = None
    for attempt in range(1, max_retries + 1):
        try:
            if use_new_sdk:
                resp = client.chat.completions.create(
                    model=model_name,
                    temperature=TEMPERATURE,
                    messages=[
                        {"role": "system", "content": "Return strict CSV only."},
                        {"role": "user", "content": prompt},
                    ],
                )
                return resp.choices[0].message.content
            else:
                import openai
                resp = openai.ChatCompletion.create(
                    model=model_name,
                    temperature=TEMPERATURE,
                    messages=[
                        {"role": "system", "content": "Return strict CSV only."},
                        {"role": "user", "content": prompt},
                    ],
                )
                return resp.choices[0].message["content"]
        except Exception as e:
            last_err = e
            time.sleep(min(2 ** attempt, 20))
    raise last_err

def csv_to_df(csv_text: str) -> pd.DataFrame:
    """Parse model CSV safely and force required columns."""
    lines = [ln.strip() for ln in str(csv_text).splitlines() if ln.strip()]
    lines = [ln for ln in lines if not ln.startswith("```")]
    text = "\n".join(lines)

    df = pd.read_csv(StringIO(text))
    df.columns = [c.strip() for c in df.columns]

    # Ensure packet_id exists
    if "packet_id" not in df.columns:
        raise ValueError("Model output missing 'packet_id' column.")

    # Ensure all expected columns exist (fill missing with empty)
    for c in OUT_COLS:
        if c not in df.columns:
            df[c] = ""

    # Keep only expected columns
    df = df[OUT_COLS].copy()

    # Normalize packet_id numeric-ish
    df["packet_id"] = pd.to_numeric(df["packet_id"], errors="coerce")

    return df

def compute_packet_metrics(gt_row: pd.Series, pred_row: pd.Series) -> dict:
    total = 0
    correct = 0
    missing = 0

    for gt_col, pred_col in FIELDS_TO_COMPARE.items():
        total += 1
        gt_val = gt_row.get(gt_col)
        pr_val = pred_row.get(pred_col)

        # normalize flags
        if gt_col == "tcp_flags":
            gt_val = norm_flags(gt_val)
            pr_val = norm_flags(pr_val)

        # normalize pkt_type
        if gt_col == "pkt_type":
            gt_val = "" if pd.isna(gt_val) else str(gt_val).upper().strip()
            pr_val = "" if pd.isna(pr_val) else str(pr_val).upper().strip()

        if pr_val is None or (isinstance(pr_val, float) and pd.isna(pr_val)) or str(pr_val).strip() == "":
            missing += 1
            continue

        if str(gt_val).strip() == str(pr_val).strip():
            correct += 1

    return {
        "fields_total": total,
        "fields_correct": correct,
        "fields_missing": missing,
        "accuracy": correct / total if total else 0.0,
        "missing_rate": missing / total if total else 0.0,
    }

def append_rows(csv_path: str, rows: list[dict]):
    """Append rows to CSV without overwriting old content."""
    if not rows:
        return
    df = pd.DataFrame(rows)
    write_header = not os.path.exists(csv_path) or os.path.getsize(csv_path) == 0
    df.to_csv(csv_path, mode="a", header=write_header, index=False)

# -------------------------
# INPUT CHECKS
# -------------------------
required_cols = ["packet_id", "hex_clean", "src_port", "dst_port", "seq", "ack", "tcp_flags", "pkt_type"]
missing_cols = [c for c in required_cols if c not in df_task.columns]
if missing_cols:
    raise ValueError(f"df_task is missing required columns: {missing_cols}")

# Work subset
df_work = (
    df_task.sort_values("packet_id")
          .head(MAX_PACKETS)
          .copy()
)
df_work["hex_clean"] = df_work["hex_clean"].apply(normalize_hex)

print("Sanity:", "packets=", len(df_work), "example_hex_len=", len(df_work["hex_clean"].iloc[0]))

# -------------------------
# RESUME SUPPORT
# -------------------------
done = set()
if os.path.exists(RAW_LOG_CSV):
    prev = pd.read_csv(RAW_LOG_CSV)
    if "key" in prev.columns:
        done = set(prev["key"].astype(str).tolist())
    print(f"Resuming: {len(done)} calls already done.")

def make_key(model, prompting, run_idx, batch_first_pid, batch_last_pid):
    return f"{model}||{prompting}||run{run_idx}||{batch_first_pid}-{batch_last_pid}"

# -------------------------
# MAIN LOOP
# -------------------------
for model in MODELS:
    for prompting in PROMPTING:
        for run_idx in range(1, N_REPETITIONS + 1):

            # batch over packets
            pids = df_work["packet_id"].astype(int).tolist()
            hexes = df_work["hex_clean"].tolist()

            for start in range(0, len(pids), BATCH_SIZE):
                end = min(start + BATCH_SIZE, len(pids))
                batch_pids = pids[start:end]
                batch_hexes = hexes[start:end]

                key = make_key(model, prompting, run_idx, batch_pids[0], batch_pids[-1])
                if key in done:
                    continue

                print(f"{model} | {prompting} | run {run_idx} | packets {batch_pids[0]}..{batch_pids[-1]}")

                prompt = build_prompt_batch(batch_pids, batch_hexes, prompting)

                raw_rows = []
                metric_rows = []

                try:
                    out_text = call_llm(prompt, model)
                    pred_df = csv_to_df(out_text)
                    status = "ok"
                    err = ""
                except Exception as e:
                    pred_df = pd.DataFrame(columns=OUT_COLS)
                    out_text = ""
                    status = "fail"
                    err = str(e)[:300]

                # Map predictions by packet_id for evaluation
                pred_map = {}
                if status == "ok":
                    pred_df = pred_df.dropna(subset=["packet_id"])
                    pred_df["packet_id"] = pred_df["packet_id"].astype(int, errors="ignore")
                    for _, r in pred_df.iterrows():
                        pid = r["packet_id"]
                        pred_map[int(pid)] = r

                # For each packet in the batch, compute metrics
                for pid in batch_pids:
                    gt_row = df_work[df_work["packet_id"].astype(int) == int(pid)].iloc[0]
                    pred_row = pred_map.get(int(pid), pd.Series({c: "" for c in OUT_COLS}))

                    m = compute_packet_metrics(gt_row, pred_row)

                    # metrics row (THIS is your requested CSV style)
                    metric_rows.append({
                        "LLM": model,
                        "Prompting method": prompting,
                        "Run": run_idx,
                        "packet_id": int(pid),
                        "accuracy": m["accuracy"],
                        "missing_rate": m["missing_rate"],
                        "fields_correct": m["fields_correct"],
                        "fields_missing": m["fields_missing"],
                        "fields_total": m["fields_total"],
                    })

                # raw log row (1 per API call batch)
                raw_rows.append({
                    "key": key,
                    "LLM": model,
                    "Prompting": prompting,
                    "Run": run_idx,
                    "packet_id_range": f"{batch_pids[0]}-{batch_pids[-1]}",
                    "status": status,
                    "error": err,
                    "model_output_preview": (out_text[:500] if out_text else ""),
                })

                append_rows(RAW_LOG_CSV, raw_rows)
                append_rows(METRICS_CSV, metric_rows)

                done.add(key)
                time.sleep(SLEEP_SEC)

print("✅ Done batch runs.")
print(f"Saved: {RAW_LOG_CSV}")
print(f"Saved: {METRICS_CSV}")




Sanity: packets= 30 example_hex_len= 112
gpt-4.1 | baseline | run 1 | packets 1..10
gpt-4.1 | baseline | run 1 | packets 11..20
gpt-4.1 | baseline | run 1 | packets 21..30
gpt-4.1 | baseline | run 2 | packets 1..10
gpt-4.1 | baseline | run 2 | packets 11..20
gpt-4.1 | baseline | run 2 | packets 21..30
gpt-4.1 | baseline | run 3 | packets 1..10
gpt-4.1 | baseline | run 3 | packets 11..20
gpt-4.1 | baseline | run 3 | packets 21..30
gpt-4.1 | oneshot | run 1 | packets 1..10
gpt-4.1 | oneshot | run 1 | packets 11..20
gpt-4.1 | oneshot | run 1 | packets 21..30
gpt-4.1 | oneshot | run 2 | packets 1..10
gpt-4.1 | oneshot | run 2 | packets 11..20
gpt-4.1 | oneshot | run 2 | packets 21..30
gpt-4.1 | oneshot | run 3 | packets 1..10
gpt-4.1 | oneshot | run 3 | packets 11..20
gpt-4.1 | oneshot | run 3 | packets 21..30
gpt-3.5-turbo | baseline | run 1 | packets 1..10
gpt-3.5-turbo | baseline | run 1 | packets 11..20
gpt-3.5-turbo | baseline | run 1 | packets 21..30
gpt-3.5-turbo | baseline | run 2 

# **3. Final Output**

In [61]:
# -------------------------
# SUMMARY
# -------------------------
dfm = pd.read_csv(METRICS_CSV)
summary = (
    dfm.groupby(["LLM", "Prompting method"], as_index=False)
       .agg(
            accuracy_mean=("accuracy", "mean"),
            accuracy_std=("accuracy", "std"),
            missing_mean=("missing_rate", "mean"),
            missing_std=("missing_rate", "std"),
            n_packets=("packet_id", "count"),
       )
)
summary.to_csv(SUMMARY_CSV, index=False)
print(f"Saved: {SUMMARY_CSV}")

Saved: metrics_summary_llm_prompting.csv
